In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Imoprting Libraries

In [ ]:
import tensorflow as tf # These lib will be useful in getting the APIs for tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import os # for creating directories, os related stuff
import math
import shutil # to shuffle the images in the dir
import random # again to shufle I guess, not sure!
import matplotlib.pyplot as plt # to plt
import seaborn as sns # to plt
from keras.preprocessing.image import load_img, img_to_array # load image from the dirs
from glob import glob #get the paths which has some pattern to follow
import cv2

In [ ]:
data = pd.read_csv('../input/lego-minifigures-classification/index.csv')
data.head(10)

In [ ]:
for path in data['path']:
    img = cv2.imread(str('../input/lego-minifigures-classification'+'/'+path)) #Just reading the path to get the very first image
    print(np.array(img)/255)
    break

In [ ]:
X = []
y = []

for path in data['path']:
    img = cv2.imread(str('../input/lego-minifigures-classification'+'/'+path))
    resized_img = cv2.resize(img,(224,224)) # Resizing the image
    scaled_img = np.array(resized_img)/255 # Scaling the image
    X.append(scaled_img) # Appending it to X

In [ ]:
y = np.array(data['class_id'])-1

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=0)

In [ ]:
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal',input_shape = (224,224,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

### Model Building

In [ ]:
num_classes = data['class_id'].nunique()

In [ ]:
model = Sequential([
    data_augmentation,
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes),
])

In [ ]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # since we did not use any activation in the output layer
    optimizer = 'adam',
    metrics = ['accuracy'],
)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,verbose=2)